## Scrape Data from Robert Koch Institut
https://experience.arcgis.com/experience/478220a4c454480e823b17327b2bf1d4/page/page_0/

Note: The RKI API only allows upto 2000 results per request. 
Therefore it is necessary to filter by country, and e.g. get the deaths and infections separatly. 

However, it may be necessary to re-adjust this when the case-numbers grow (when there are more than 2000 rows for infections in Bavaria)

In [ ]:
import urllib
import json
import datetime
import pandas

In [ ]:
# Just a list of all countries
bundeslaender = ["Baden-Württemberg","Nordrhein-Westfalen","Bayern","Hessen","Berlin",
                 "Niedersachsen","Sachsen","Rheinland-Pfalz","Brandenburg","Hamburg","Schleswig-Holstein"
                ,"Thüringen","Mecklenburg-Vorpommern","Bremen","Saarland","Sachsen-Anhalt"]

## Fetching Infection Numbers

This part of the notebook fetches the data of the RKI

In [ ]:
safe_infection_file = True
infection_file_name = "Neuinfektionen_pro_land_pro_tag.csv"

In [ ]:
def fetch_infection_data_from_rki(bundesland:str="Hamburg",offset=0):
    """
    Fetch Covid-19-Cases from 
    https://experience.arcgis.com/experience/478220a4c454480e823b17327b2bf1d4/page/page_0/
    
    Args:
        bundesland: written like displayed on the website, a string
    Returns:
        a Dataframe containing all historical infections data of a bundesland
    """
    
    url_endpoint = "https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_COVID19/FeatureServer/0/query"
    params = {
        'f': 'json', 
        'where': f'Bundesland=\'{bundesland}\'',
        'returnGeometry': 'false',
        'spatialRel': 'esriSpatialRelIntersects',
        'outFields': 'ObjectId,AnzahlFall,Meldedatum,Geschlecht,Altersgruppe',
        'orderByFields': 'Meldedatum asc',
        'resultOffset': offset,
        'resultRecordCount': 2000,
        'cacheHint': "true"    
    }

    url_query = f"{url_endpoint}?{urllib.parse.urlencode(params)}"

    with urllib.request.urlopen(url_query) as url:
        data = json.loads(url.read().decode())['features']
    
    data_list = [
        (datetime.datetime.fromtimestamp(x['attributes']['Meldedatum'] / 1e3), x['attributes']['AnzahlFall'],x['attributes']['Geschlecht'],x['attributes']['Altersgruppe'],bundesland) 
        for x in data
    ]
    
    df = pandas.DataFrame(data_list, columns=['Meldedatum', 'Neuinfektionen', 'Geschlecht','Altersgruppe','Bundesland'])

    if len(data_list)>= 2000:
        df = df.append(fetch_infection_data_from_rki(bundesland,offset+2000))
    
    return df

Sample Run for Bayern

In [ ]:
df = fetch_infection_data_from_rki("Bayern")
df.head()

aggregated Run for all countries

In [ ]:
%%time
# get the first country data in a dataframe
all_country_infection_data = fetch_infection_data_from_rki(bundeslaender[0])
# append all other rows to it
for bland in bundeslaender[1:]:
    all_country_infection_data=all_country_infection_data.append(fetch_infection_data_from_rki(bland))

In [ ]:
aggregated_infection_country_data=all_country_infection_data.groupby(["Bundesland","Meldedatum","Geschlecht","Altersgruppe"]).aggregate(sum)

In [ ]:
# To look at 
aggregated_infection_country_data

In [ ]:
if safe_infection_file:
    aggregated_infection_country_data.to_csv(infection_file_name)

## Fetching Death Numbers :(

In [ ]:
safe_death_file = True
death_file_name = "Todesfaelle_pro_land_pro_tag.csv"

In [ ]:
def fetch_death_data_from_rki(bundesland:str="Hamburg",offset=0):
    """
    Fetch Covid-19-Cases from 
    https://experience.arcgis.com/experience/478220a4c454480e823b17327b2bf1d4/page/page_0/
    
    Args:
        bundesland: written like displayed on the website, a string
    Returns:
        a Dataframe containing all historical infections data of a bundesland
    """
    
    url_endpoint = "https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_COVID19/FeatureServer/0/query"
    params = {
        'f': 'json', 
        'where': f'Bundesland=\'{bundesland}\' AND AnzahlTodesfall>0',
        'returnGeometry': 'false',
        'spatialRel': 'esriSpatialRelIntersects',
        'outFields': 'ObjectId,AnzahlTodesfall,Meldedatum,Geschlecht,Altersgruppe',
        'orderByFields': 'Meldedatum asc',
        'resultOffset': offset,
        'resultRecordCount': 2000,
        'cacheHint': "true"    
    }

    url_query = f"{url_endpoint}?{urllib.parse.urlencode(params)}"

    with urllib.request.urlopen(url_query) as url:
        data = json.loads(url.read().decode())['features']
    
    data_list = [
        (datetime.datetime.fromtimestamp(x['attributes']['Meldedatum'] / 1e3), x['attributes']['AnzahlTodesfall'],x['attributes']['Geschlecht'],x['attributes']['Altersgruppe'],bundesland) 
        for x in data
    ]
    
    df = pandas.DataFrame(data_list, columns=['Meldedatum', 'Todesfaelle', 'Geschlecht','Altersgruppe','Bundesland'])

    if len(data_list)>= 2000:
        df = df.append(fetch_death_data_from_rki(bundesland,offset+2000))
    
    return df

In [ ]:
df = fetch_death_data_from_rki("Bayern")
df.head()

Aggregated run for all countries 

In [ ]:
%%time
# get the first country data in a dataframe
all_country_death_data = fetch_death_data_from_rki(bundeslaender[0])
# append all other rows to it
for bland in bundeslaender[1:]:
    all_country_death_data=all_country_death_data.append(fetch_death_data_from_rki(bland))
    
aggregated_death_country_data=all_country_death_data.groupby(["Bundesland","Meldedatum","Geschlecht","Altersgruppe"]).aggregate(sum)

In [ ]:
aggregated_death_country_data

In [ ]:
if safe_death_file:
    aggregated_death_country_data.to_csv(death_file_name)

## Pivot Tables 

As per discussion, for the merged file of morgenpost and rki we want to have rows per day showing the newly infected per day, with columns for the countries

e.g. 

Day | Deaths Bayern | Infects Bayern | Deaths BaWü | Infects BaWü | ...

In [ ]:
high_level_aggregated_infection_data = all_country_infection_data.groupby(["Meldedatum","Bundesland"])

In [ ]:
# Example how to access the 28th January for bayern
high_level_aggregated_infection_data.get_group(("2020-01-28","Bayern")).sum()

In [ ]:
# Really only get the value (no clutter)
high_level_aggregated_infection_data.get_group(("2020-01-28","Bayern")).sum()[0]

In [ ]:
infections_dates = set(all_country_infection_data.Meldedatum.unique())
death_dates =  set(all_country_death_data.Meldedatum.unique())
all_dates = list(infections_dates.union(death_dates))
all_dates.sort()

In [ ]:
#all_dates

In [ ]:
# Example how to access the 28th January for bayern with the requested dates
high_level_aggregated_infection_data.get_group((all_dates[1],"Bayern")).sum()

In [ ]:
# Simple error catching for empty entries
# Change Bayern to Hessen to see the error

try:
    print(high_level_aggregated_infection_data.get_group((all_dates[1],"Bayern")).sum())
except(KeyError):
    print(0)

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]

def get_all_dates_sorted(all_death_data,all_infection_data):
    infections_dates = set(all_infection_data.Meldedatum.unique())
    death_dates =  set(all_death_data.Meldedatum.unique())
    all_dates = list(infections_dates.union(death_dates))
    all_dates.sort()
    return all_dates

def get_pivoted_country_data(all_death_data,all_infection_data):
    dates = get_all_dates_sorted(all_death_data,all_infection_data)
    bundeslaender = ["Baden-Württemberg","Nordrhein-Westfalen","Bayern","Hessen","Berlin",
                 "Niedersachsen","Sachsen","Rheinland-Pfalz","Brandenburg","Hamburg","Schleswig-Holstein"
                ,"Thüringen","Mecklenburg-Vorpommern","Bremen","Saarland","Sachsen-Anhalt"]

    grouped_infection_data = all_infection_data.groupby(["Meldedatum","Bundesland"])
    grouped_death_data = all_death_data.groupby(["Meldedatum","Bundesland"])
    
    data = []
    for date in dates:
        row = [date]
        for bland in bundeslaender:
            try:
                i_value = grouped_infection_data.get_group((date,bland)).sum()
                row= row +[i_value['Neuinfektionen']]
            except(KeyError):
                row= row +[0]
            try:
                i_value = grouped_death_data.get_group((date,bland)).sum()
                row= row +[i_value['Todesfaelle']]
            except(KeyError):
                row= row +[0]
        data = data + [row]
    
    columns = ["Datum"]
    columns = columns + flatten([[f"RKI:Infektionen:{bland}",f"RKI:Tode:{bland}"] for bland in bundeslaender])
    
    df = pandas.DataFrame(data,columns=columns)
    
    #Todo: Aggregate all values to the date they have, 
    # So that it's not "new Infections at Day X" but "total infections at Day x"
    
    return df

In [ ]:
get_pivoted_country_data(all_country_death_data,all_country_infection_data)